# Tutorial for Encryption and Decryption Features in MLCBase

[![PyPI](https://img.shields.io/pypi/v/mlcbase)](https://pypi.org/project/mlcbase/) &nbsp;
[![license](https://img.shields.io/github/license/wmchen/mlcbase.svg)](https://www.apache.org/licenses/LICENSE-2.0)

Author: [Weiming Chen](https://weimingchen.net) and [Yuanshuang Sun](https://www.mulingcloud.com/author/yuanshuang-sun/)

## Introduction

We offer various methods to encrypt and decrypt or verify text, files and passwords including RSA, AES and Hash.

In [1]:
import sys
sys.path.append("../src")
from datetime import datetime
from mlcbase import (Logger, create_rsa_keys, rsa_encrypt_text, rsa_decrypt_text, rsa_sign_text,
                     rsa_verify_signature, rsa_encrypt_file, rsa_decrypt_file, aes_encrypt_text, 
                     aes_decrypt_text, aes_entrypt_file, aes_decrypt_file, encrypt_password, 
                     verify_password, wrap_module_timer, runtime_analysis, delete_register_modules, 
                     random_hex)

logger = Logger()
logger.init_logger()


👋 Welcome to use MuLingCloud. We aim to let everything easier.

📍 mlcbase (1.2.0.dev.202405) imported



## 1. RSA

### 1.1 Create RSA keys

You can create a pair of RSA public and private keys by calling `create_rsa_keys()`.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `public_path` | Optional[PathLikeType] | The path you want to save the public key. Defaults to None |
| `private_path` | Optional[PathLikeType] | The path you want to save the private key. Defaults to None |
| `key_length` | int | The length of key. Defaults to 2048 |
| `return_keys` | bool | Defaults to True |

We force the `key_length` must be larger or equal to 2048 for safety resons.

Common options of `key_length` including 2048, 3072, and 4096.

#### Return

It returns a pair of RSA public and private keys in tuple i.e. `(public_key, private_key)` if `return_keys` is True, otherwise returns None.

In [2]:
create_rsa_keys(public_path="./examples/public.pem", private_path="./examples/private.pem")

Besides, you can set `return_keys=True` to return the keys.

### 1.2 Encrypt a plain text

You can use the public key to encrypt a plain text by calling `rsa_encrypt_text()`.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `plain_text` | Union[str, bytes] | The plain text waiting for encryption, which can be the type of `str` or `bytes` |
| `public_key` | Union[bytes, PathLikeType] | The public key |
| `key_length` | int | The length of key. Defaults to 2048 |
| `num_threads` | int | The number of threads to use. Defaults to 1 |
| `encoding` | str | The encoding method. Defaults to "utf-8" |

The `public_key` can be the path of the public key, and also can be the content of the public key in bytes.

If `num_threads` larger than 1, it will use multithreading to accelerate the encryption.

However, the performance of multithreading is far from expectation due to the GIL of Python.

#### Return

It returns the cipher text in bytes if the plain text in bytes is shorter than `key_length // 8 - 11`, otherwise returns the cipher text in list.

In [3]:
plain_text = "Hello world! Welcome to use MuLingCloud. We aim to let everthing become easier."
cipher_text = rsa_encrypt_text(plain_text, "./examples/public.pem")
logger.info(type(cipher_text))

2024-05-04 22:23:00 | 0 day(s) 00:00:00 | INFO | <class 'bytes'>


If the plain text in bytes is longer than `key_length // 8 - 11`, it will be divided into several pieces and encrypted separately. Then, returns a list of cipher texts.

In [4]:
long_text = 10 * plain_text
cipher_long_text = rsa_encrypt_text(long_text, "./examples/public.pem")
logger.info(type(cipher_long_text))

2024-05-04 22:23:00 | 0 day(s) 00:00:00 | INFO | <class 'list'>


### 1.3 Decrypt a cipher text

You can use the private key to decrypt a cipher text by calling `rsa_decrypt_text()`.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `cipher_text` | Union[List[bytes], bytes] | The cipher text waiting for decryption, which can be the type of `bytes` or a list of `bytes` |
| `private_key` | Union[bytes, PathLikeType] | The private key |
| `key_length` | int | The length of key. Defaults to 2048 |
| `num_threads` | int | The number of threads to use. Defaults to 1 |
| `return_str` | bool | Whether to return in `str`. Defaults to True |
| `encoding` | str | The encoding method. Defaults to "utf-8" |

The `private_key` can be the path of the private key, and also can be the content of the private key in bytes.

If `num_threads` larger than 1, it will use multithreading to accelerate the decryption.

However, the performance of multithreading is far from expectation due to the GIL of Python.

#### Return

It returns the plain text in `str` if `return_str` is True, otherwise returns the plain text in `bytes`.

In [5]:
decrypted_pain_text = rsa_decrypt_text(cipher_text, "./examples/private.pem")
if decrypted_pain_text == plain_text:
    logger.success("Decrypted accurately")

2024-05-04 22:23:00 | 0 day(s) 00:00:00 | SUCCESS | Decrypted accurately


The `cipher_text` also can be a list of `bytes`.

In [6]:
decrypted_long_text = rsa_decrypt_text(cipher_long_text, "./examples/private.pem")
if decrypted_long_text == long_text:
    logger.success("Decrypted accurately")

2024-05-04 22:23:00 | 0 day(s) 00:00:00 | SUCCESS | Decrypted accurately


It is worth to mention that it is unefficient to use RSA encryption and decryption for a extreme long text.

But you can set `num_threads` to specify the number of threads to speed up the process to some extent.

However, due to the existence of GIL of Python, the performance of multithreading is far from expectation. 

In [7]:
extreme_long_text = 10 * 1024 * plain_text

start_time = datetime.now()
cipher_extreme_long_text = rsa_encrypt_text(extreme_long_text, "./examples/public.pem", num_threads=1)
decrypted_extreme_long_text = rsa_decrypt_text(cipher_extreme_long_text, "./examples/private.pem", num_threads=1)
if decrypted_extreme_long_text == extreme_long_text:
    logger.success("Decrypted accurately")
end_time = datetime.now()
logger.info(f"Elapsed without multithreading: {(end_time - start_time).total_seconds()}")

start_time = datetime.now()
cipher_extreme_long_text = rsa_encrypt_text(extreme_long_text, "./examples/public.pem", num_threads=8)
decrypted_extreme_long_text = rsa_decrypt_text(cipher_extreme_long_text, "./examples/private.pem", num_threads=8)
if decrypted_extreme_long_text == extreme_long_text:
    logger.success("Decrypted accurately")
end_time = datetime.now()
logger.info(f"Elapsed with multithreading (num_threads=8): {(end_time - start_time).total_seconds()}")

2024-05-04 22:23:16 | 0 day(s) 00:00:16 | SUCCESS | Decrypted accurately
2024-05-04 22:23:16 | 0 day(s) 00:00:16 | INFO | Elapsed without multithreading: 16.042685
2024-05-04 22:23:32 | 0 day(s) 00:00:32 | SUCCESS | Decrypted accurately
2024-05-04 22:23:32 | 0 day(s) 00:00:32 | INFO | Elapsed with multithreading (num_threads=8): 15.242871


### 1.4 Sign a text

Another important feature of RSA is to sign and verify the text.

You can sign a plain text with the private key by calling `rsa_sign_text()` to get the corresponding signature.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `plain_text` | str | The plain text waiting for signing |
| `private_key` | Union[bytes, PathLikeType] | The private key |
| `hash_method` | str | The hash method to use. Defaults to "SHA-512" |
| `encoding` | str | The encoding method. Defaults to "utf-8" |

The `private_key` can be the path of the private key, and also can be the content of the private key in bytes.

The options of `hash_method` including "MD5", "SHA-1", "SHA-224", "SHA-256", "SHA-384", and "SHA-512".

#### Return

It returns the signature in bytes.

In [8]:
plain_text = "This is a message with sensitive information."
signature = rsa_sign_text(plain_text, "./examples/private.pem")
logger.info(type(signature))

2024-05-04 22:23:32 | 0 day(s) 00:00:32 | INFO | <class 'bytes'>


Note that the signature is undecryptable.

In [9]:
try:
    rsa_decrypt_text(signature, "./examples/private.pem")
    logger.success("Decrypted")
except Exception as e:
    logger.error(str(e))

2024-05-04 22:23:32 | 0 day(s) 00:00:32 | ERROR | Decryption failed


### 1.5 Verify a signature

You can use the public key to verify if a signature is match to a plain text by calling `rsa_verify_signature()`.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `plain_text` | str | The plain text |
| `signature` | bytes | The signature |
| `public_key` | Union[bytes, PathLikeType] | The public key |
| `encoding` | str | The encoding method. Defaults to "utf-8" |

The `public_key` can be the path of the public key, and also can be the content of the public key in bytes.

#### Return

It returns True if the signature match to the plain text, otherwise returns False.

In [10]:
rsa_verify_signature(plain_text, signature, "./examples/public.pem")

True

### 1.6 Encrypt a file

You can encrypt a file with the public key by calling `rsa_encrypt_file()`.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `plain_file_path` | PathLikeType | The path of the plain file |
| `crypto_save_path` | PathLikeType | The save path of the crypto |
| `public_key` | Union[bytes, PathLikeType] | The public key |
| `key_length` | int | The length of key. Defaults to 2048 |
| `num_process` | int | The number of processes to use. Defaults to 1 |
| `num_threads` | int | The number of threads to use. Defaults to 1 |
| `encoding` | str | The encoding method. Defaults to "utf-8" |
| `logger` | Optional[Logger] | Defaults to None |

The `public_key` can be the path of the public key, and also can be the content of the public key in bytes.

If `num_process` larger than 1, it will use multiprocessing to accelerate the encryption (which is effective).

If `num_threads` larger than 1, it will use multithreading to accelerate the encryption.

However, the performance of multithreading is far from expectation due to the GIL of Python.

#### Return

It returns True if success, otherwise return False.

In [11]:
rsa_encrypt_file(plain_file_path="./examples/jsonfile.json",
                 crypto_save_path="./examples/jsonfile.rsa_encrypted.bin",
                 public_key="./examples/public.pem",
                 logger=logger)

True

### 1.7 Decrypt a file

You can decrypt a file with the private key by calling `rsa_decrypt_file()`.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `crypto_file_path` | PathLikeType | The path of the crypto |
| `plain_save_path` | PathLikeType | The save path of the plain file |
| `private_key` | Union[bytes, PathLikeType] | The private key |
| `key_length` | int | The length of key. Defaults to 2048 |
| `num_process` | int | The number of processes to use. Defaults to 1 |
| `num_threads` | int | The number of threads to use. Defaults to 1 |
| `encoding` | str | The encoding method. Defaults to "utf-8" |
| `logger` | Optional[Logger] | Defaults to None |

The `private_key` can be the path of the private key, and also can be the content of the private key in bytes.

If `num_process` larger than 1, it will use multiprocessing to accelerate the decryption (which is effective).

If `num_threads` larger than 1, it will use multithreading to accelerate the decryption.

However, the performance of multithreading is far from expectation due to the GIL of Python.

#### Return

It returns True if success, otherwise return False.

In [12]:
rsa_decrypt_file(crypto_file_path="./examples/jsonfile.rsa_encrypted.bin",
                 plain_save_path="./examples/jsonfile.rsa_decrypted.json",
                 private_key="./examples/private.pem",
                 logger=logger)

True

RSA encryption and decryption is **NOT RECOMMENDED** for large files!

If you really want to use RSA to encrypt and decrypt large files, we offer multiprocessing and multithreading to speed up the process.

You can set `num_process` to specify the number of process to speed up the enryption and decryption process, while setting `num_threads` to further gain a little acceleration.

In the following example, we set `num_process=8` and `num_threads=8` and gain a significant acceleration of the entire process.

In [13]:
@wrap_module_timer
def rsa_encrypt_large_file():
    rsa_encrypt_file(plain_file_path="./examples/YOLOv9.pdf",
                     crypto_save_path="./examples/YOLOv9.rsa_encrypted.bin",
                     public_key="./examples/public.pem",
                     logger=logger)
    
@wrap_module_timer
def rsa_decrypt_large_file():
    rsa_decrypt_file(crypto_file_path="./examples/YOLOv9.rsa_encrypted.bin",
                     plain_save_path="./examples/YOLOv9.rsa_decrypted.pdf",
                     private_key="./examples/private.pem",
                     logger=logger) 

@wrap_module_timer
def rsa_encrypt_large_file_accelerate():
    rsa_encrypt_file(plain_file_path="./examples/YOLOv9.pdf",
                     crypto_save_path="./examples/YOLOv9.rsa_encrypted.bin",
                     public_key="./examples/public.pem",
                     num_process=8,
                     num_threads=8,
                     logger=logger)
    
@wrap_module_timer
def rsa_decrypt_large_file_accelerate():
    rsa_decrypt_file(crypto_file_path="./examples/YOLOv9.rsa_encrypted.bin",
                     plain_save_path="./examples/YOLOv9.rsa_decrypted.pdf",
                     private_key="./examples/private.pem",
                     num_process=8,
                     num_threads=8,
                     logger=logger)
    
rsa_encrypt_large_file()
rsa_decrypt_large_file()
rsa_encrypt_large_file_accelerate()
rsa_decrypt_large_file_accelerate()
runtime_analysis(unit="s")

+-------+-----------------------------------+-------------+-------+-----------------+
| index |               module              | elapsed (s) | calls | avg_runtime (s) |
+-------+-----------------------------------+-------------+-------+-----------------+
|   1   |       rsa_decrypt_large_file      |   104.408   |   1   |     104.408     |
|   2   | rsa_decrypt_large_file_accelerate |    12.657   |   1   |      12.657     |
|   3   |       rsa_encrypt_large_file      |    2.216    |   1   |      2.216      |
|   4   | rsa_encrypt_large_file_accelerate |    1.014    |   1   |      1.014      |
+-------+-----------------------------------+-------------+-------+-----------------+


**HOWEVER**, note that the creation and destruction of processes will cost unignorable time, which means multiprocessing cannot not always accelerate the entire process.

In the following example, we set `num_process=8` and `num_threads=8` but slow down the entire process.

In [14]:
@wrap_module_timer
def rsa_encrypt_small_file():
    rsa_encrypt_file(plain_file_path="./examples/jsonfile.json",
                     crypto_save_path="./examples/jsonfile.rsa_encrypted.bin",
                     public_key="./examples/public.pem",
                     logger=logger)
    
@wrap_module_timer
def rsa_decrypt_small_file():
    rsa_decrypt_file(crypto_file_path="./examples/jsonfile.rsa_encrypted.bin",
                     plain_save_path="./examples/jsonfile.rsa_decrypted.json",
                     private_key="./examples/private.pem",
                     logger=logger) 

@wrap_module_timer
def rsa_encrypt_small_file_accelerate():
    rsa_encrypt_file(plain_file_path="./examples/jsonfile.json",
                     crypto_save_path="./examples/jsonfile.rsa_encrypted.bin",
                     public_key="./examples/public.pem",
                     num_process=8,
                     num_threads=8,
                     logger=logger)
    
@wrap_module_timer
def rsa_decrypt_small_file_accelerate():
    rsa_decrypt_file(crypto_file_path="./examples/jsonfile.rsa_encrypted.bin",
                     plain_save_path="./examples/jsonfile.rsa_decrypted.json",
                     private_key="./examples/private.pem",
                     num_process=8,
                     num_threads=8,
                     logger=logger)
    
rsa_encrypt_small_file()
rsa_decrypt_small_file()
rsa_encrypt_small_file_accelerate()
rsa_decrypt_small_file_accelerate()
runtime_analysis(unit="s")

+-------+-----------------------------------+-------------+-------+-----------------+
| index |               module              | elapsed (s) | calls | avg_runtime (s) |
+-------+-----------------------------------+-------------+-------+-----------------+
|   1   |       rsa_decrypt_large_file      |   104.408   |   1   |     104.408     |
|   2   | rsa_decrypt_large_file_accelerate |    12.657   |   1   |      12.657     |
|   3   |       rsa_encrypt_large_file      |    2.216    |   1   |      2.216      |
|   4   | rsa_encrypt_large_file_accelerate |    1.014    |   1   |      1.014      |
|   5   | rsa_decrypt_small_file_accelerate |    0.745    |   1   |      0.745      |
|   6   | rsa_encrypt_small_file_accelerate |    0.704    |   1   |      0.704      |
|   7   |       rsa_decrypt_small_file      |    0.011    |   1   |      0.011      |
|   8   |       rsa_encrypt_small_file      |    0.001    |   1   |      0.001      |
+-------+-----------------------------------+---------

## 2. AES

We currently only support `CBC` and `ECB` modes. But the `ECB` mode is not safe enough, we recommend more to use the `CBC` mode in production environment.

For `CBC` mode, both `key` and `iv` are needed.

For `ECB` mode, only the `key` is needed.

The `key` is a hex string which length should be 16 (AES128), 24 (AES192), or 32 (AES256).

The `iv` is a hex string which length should be 16.

We only show the usage of the default `CBC` mode in the following example.

In [15]:
key = random_hex(24)
iv = random_hex(16)

### 2.1 Encrypt a plain text

You can encrypt a plain text by calling `aes_encrypt_text()`.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `plain_text` | str | The plain text |
| `key` | Union[str, bytes] | The secret key  |
| `iv` | Optional[Union[str, bytes]] | The initialization vector. Defaults to None |
| `mode` | int | The AES mode. Defaults to AES.MODE_CBC |
| `encoding` | str | The encoding method. Defaults to "utf-8" |

The initialization vector `iv` is required when `mode=AES.MODE_CBC`, and the length of `iv` should be 16.

#### Return

It returns the cipher text in bytes.

In [16]:
plain_text = "Hello world! Welcome to use MuLingCloud. We aim to let everthing become easier."
cipher_text = aes_encrypt_text(plain_text, key, iv)
logger.info(type(cipher_text))

2024-05-04 22:25:34 | 0 day(s) 00:02:34 | INFO | <class 'bytes'>


### 2.2 Decrypt a cipher text

You can decrypt a cipher text by calling `aes_decrypt_text()`.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `cipher_text` | bytes | The cipher text |
| `key` | Union[str, bytes] | The secret key  |
| `iv` | Optional[Union[str, bytes]] | The initialization vector. Defaults to None |
| `mode` | int | The AES mode. Defaults to AES.MODE_CBC |
| `return_str` | bool | Whether to return in `str`. Defaults to True |
| `encoding` | str | The encoding method. Defaults to "utf-8" |

The initialization vector `iv` is required when `mode=AES.MODE_CBC`, and the length of `iv` should be 16.

#### Return

It returns the plain text in `str` if `return_str` is True, otherwise returns the plain text in `bytes`.

In [17]:
decrypted_pain_text = aes_decrypt_text(cipher_text, key, iv)
if decrypted_pain_text == plain_text:
    logger.success("Decrypted accurately")

2024-05-04 22:25:34 | 0 day(s) 00:02:34 | SUCCESS | Decrypted accurately


### 2.3 Encrypt a file

You can encrypt a file by calling `aes_entrypt_file()`.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `plain_file_path` | PathLikeType | The path of the plain file |
| `crypto_save_path` | PathLikeType | The save path of the crypto |
| `key` | Union[str, bytes] | The secret key  |
| `iv` | Optional[Union[str, bytes]] | The initialization vector. Defaults to None |
| `mode` | int | The AES mode. Defaults to AES.MODE_CBC |
| `encoding` | str | The encoding method. Defaults to "utf-8" |
| `logger` | Optional[Logger] | Defaults to None |

The initialization vector `iv` is required when `mode=AES.MODE_CBC`, and the length of `iv` should be 16.

#### Return

It returns True if success, otherwise return False.

In [18]:
aes_entrypt_file(plain_file_path="./examples/jsonfile.json",
                 crypto_save_path="./examples/jsonfile.aes_encrypted.bin",
                 key=key,
                 iv=iv)

True

### 2.4 Decrypt a file

You can decrypt a file by calling `aes_decrypt_file()`.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `crypto_file_path` | PathLikeType | The path of the crypto |
| `plain_save_path` | PathLikeType | The save path of the plain file |
| `key` | Union[str, bytes] | The secret key  |
| `iv` | Optional[Union[str, bytes]] | The initialization vector. Defaults to None |
| `mode` | int | The AES mode. Defaults to AES.MODE_CBC |
| `encoding` | str | The encoding method. Defaults to "utf-8" |
| `logger` | Optional[Logger] | Defaults to None |

The initialization vector `iv` is required when `mode=AES.MODE_CBC`, and the length of `iv` should be 16.

#### Return

It returns True if success, otherwise return False.

In [19]:
aes_decrypt_file(crypto_file_path="./examples/jsonfile.aes_encrypted.bin",
                 plain_save_path="./examples/jsonfile.aes_decrypted.json",
                 key=key,
                 iv=iv)

True

We highly recommend you to use AES to encrypt and decrypt files, which is way more efficient than RSA.

**In practical applications, we recommend you to use AES to encrypt and decrypt files and use RSA to encrypt and decrypt the AES key and iv.**

In [20]:
@wrap_module_timer
def aes_encrypt_large_file():
    aes_entrypt_file(plain_file_path="./examples/YOLOv9.pdf",
                     crypto_save_path="./examples/YOLOv9.pdf.aes_encrypted.bin",
                     key=key,
                     iv=iv)
    
@wrap_module_timer
def aes_decrypt_large_file():
    aes_decrypt_file(crypto_file_path="./examples/YOLOv9.pdf.aes_encrypted.bin",
                     plain_save_path="./examples/YOLOv9.pdf.aes_decrypted.pdf",
                     key=key,
                     iv=iv)
    
aes_encrypt_large_file()
aes_decrypt_large_file()
runtime_analysis(unit="s")

+-------+-----------------------------------+-------------+-------+-----------------+
| index |               module              | elapsed (s) | calls | avg_runtime (s) |
+-------+-----------------------------------+-------------+-------+-----------------+
|   1   |       rsa_decrypt_large_file      |   104.408   |   1   |     104.408     |
|   2   | rsa_decrypt_large_file_accelerate |    12.657   |   1   |      12.657     |
|   3   |       rsa_encrypt_large_file      |    2.216    |   1   |      2.216      |
|   4   | rsa_encrypt_large_file_accelerate |    1.014    |   1   |      1.014      |
|   5   | rsa_decrypt_small_file_accelerate |    0.745    |   1   |      0.745      |
|   6   | rsa_encrypt_small_file_accelerate |    0.704    |   1   |      0.704      |
|   7   |       aes_encrypt_large_file      |    0.019    |   1   |      0.019      |
|   8   |       rsa_decrypt_small_file      |    0.011    |   1   |      0.011      |
|   9   |       aes_decrypt_large_file      |    0.008

## 3. Password

We offer a simple script to encrypt and verify the password with hash algorithms.

### 3.1 Encrypt a plain password

You can encrypt a password by calling `encrypt_password()`.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `password` | str | The plain password |
| `methods` | Union[str, List[str]] | The encrypting methods |
| `encoding` | str | The encoding method. Defaults to "utf-8" |

The encrypting methods should be a `str` or a list of `str`. And the order of hash algorithms is following the order of `methods`.

#### Return

It returns the cipher in `str`.

As the following example, the password is encrypted by: `password -> SHA-256 -> SHA-384 -> SHA-512 -> SHA-224 -> MD5 -> cipher`.

In [21]:
password = "B0Go4P8nuQ8DQxJDUWzq"
methods = ["SHA-256", "SHA-384", "SHA-512", "SHA-224", "MD5"]
cipher = encrypt_password(password, methods)

### 3.2 Verify a cipher

You can verify if a plain password is match to a cipher by calling `verify_password()`.

#### Arguments

| args | type | remark |
| :--- | :--- | :----- |
| `password` | str | The plain password |
| `cipher` | str | The cipher |
| `methods` | Union[str, List[str]] | The encrypting methods |
| `encoding` | str | The encoding method. Defaults to "utf-8" |

The encrypting methods should be a `str` or a list of `str`. And the order of hash algorithms is following the order of `methods`.

#### Return

It returns True if the password match with the cipher, otherwise returns False.

In [22]:
verify_password(password, cipher, methods)

True